<a href="https://colab.research.google.com/github/regevDavid/Machine-Learning/blob/master/Nueral%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torch
!pip install soundfile
!pip install librosa
!git clone https://github.com/orsanawwad/ML4_dataset.git

In [0]:
!python gcommand_loader.py

python3: can't open file 'gcommand_loader.py': [Errno 2] No such file or directory


In [0]:
from gcommand_loader import GCommandLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gcommand_loader import GCommandLoader
import torch
import numpy as np

In [0]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self._conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        self._conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1, padding=2)
        self._conv3 = nn.Conv2d(in_channels=50, out_channels=100, kernel_size=5, stride=1, padding=2)
        self._conv4 = nn.Conv2d(in_channels=100, out_channels=200, kernel_size=5, stride=1, padding=2)
        self._conv5 = nn.Conv2d(in_channels=200, out_channels=400, kernel_size=5, stride=1, padding=2)

        self._dropout = nn.Dropout(p=0.5)
        self._l1 = nn.Linear(400 * 5 * 3, 30)
        self._loss_function = nn.CrossEntropyLoss()
        self._optimizer = optim.Adam(self.parameters(), lr=5e-4)

    def forward(self, x):
        x = Variable(x)
        if torch.cuda.is_available():
            x = x.cuda()
        x = self._conv1(x)
        x = functional.relu(x)
        x = functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self._dropout(x)
        x = self._conv2(x)
        x = functional.relu(x)
        x = functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self._dropout(x)
        x = self._conv3(x)
        x = functional.relu(x)
        x = functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self._dropout(x)
        x = self._conv4(x)
        x = functional.relu(x)
        x = functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self._dropout(x)
        x = self._conv5(x)
        x = functional.relu(x)
        x = functional.max_pool2d(x, kernel_size=2, stride=2)
        x = self._dropout(x)
        x = x.view(x.size(0), -1)
        x = self._l1(x)
        return x

    def train_example(self, vectors_batch, labels_batch):
        vectors_batch = Variable(vectors_batch)
        labels_batch = Variable(labels_batch)
        if torch.cuda.is_available():
            vectors_batch = vectors_batch.cuda()
            labels_batch = labels_batch.cuda()
        self._optimizer.zero_grad()
        ys = self(vectors_batch)
        loss = self._loss_function(ys, labels_batch)
        loss.backward()
        self._optimizer.step()


In [0]:
    
import os

import torch.cuda
import torch.utils.data
import tqdm
from torch.autograd import Variable

from gcommand_loader import GCommandLoader
from network import Network

TRAINING_BATCH_SIZE = 100
TESTING_BATCH_SIZE = 100

NUMBER_OF_EPOCHS = 20
ACCURACY_CHECK_SIZE = 5000


def main():
  model = Network()
  if torch.cuda.is_available():
    model.cuda()
  train_loader = torch.utils.data.DataLoader(GCommandLoader('./data/train'), batch_size=TRAINING_BATCH_SIZE, shuffle=True,
                                             num_workers=4, pin_memory=True, sampler=None)

  validation_loader = torch.utils.data.DataLoader(GCommandLoader('./data/valid'), batch_size=TESTING_BATCH_SIZE, shuffle=True,
                                                  num_workers=4, pin_memory=True, sampler=None)

  gc_test_loader = GCommandLoader('./data/test')
  test_loader = torch.utils.data.DataLoader(gc_test_loader, batch_size=TESTING_BATCH_SIZE, shuffle=False,
                                            num_workers=4, pin_memory=True, sampler=None)

  for i in range(NUMBER_OF_EPOCHS):
      print("{}/{}".format(i+1, NUMBER_OF_EPOCHS))
      model.train()
      for vector_batch, label_batch in tqdm.tqdm(train_loader, total=len(train_loader), unit_scale=TRAINING_BATCH_SIZE):
          model.train_example(vector_batch, label_batch)
      model.eval()
      print(evaluate(model, train_loader))
      print(evaluate(model, validation_loader))

  model.eval()
  all_predictions = []
  for vectors_batch, _ in test_loader:
      outputs = model(vectors_batch)
      _, predictions = torch.max(outputs.data, 1)
      all_predictions.extend(predictions)

  with open("test_y", "w") as f:
      for spect, prediction in zip(gc_test_loader.spects, all_predictions):
          f.write("{}, {}".format(os.path.basename(spect[0]), str(prediction.item())))
          f.write(os.linesep)


def evaluate(model, loader):
  total = 0
  correct = 0

  for vectors, labels in loader:
      labels = Variable(labels)
      if torch.cuda.is_available():
          labels = labels.cuda()
      outputs = model(vectors)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      if total >= ACCURACY_CHECK_SIZE:
          return correct / total

if __name__ == '__main__':
  main()

IndentationError: ignored

In [0]:
def get_data():
    train_dataset = GCommandLoader('./ML4_dataset/data/train')
    train_data = torch.utils.data.DataLoader(
        train_dataset, batch_size=100, shuffle=True,
        num_workers=20, pin_memory=True, sampler=None)

    valid_dataset = GCommandLoader('./ML4_dataset/data/valid')
    validation_data = torch.utils.data.DataLoader(
        valid_dataset, batch_size=100, shuffle=True,
        num_workers=20, pin_memory=True, sampler=None)

    return train_data, validation_data

In [0]:
def test_data(data, model):
    counter = 0
    for x, y in data:
        print(y)
        y_pred = np.argmax(list(model.forward(x)))
        print(y_pred)
        y = int(y.astype(np.float))
        if y == y_pred:
            counter += 1
    print("accuracy is : " + str(counter / len(data)))

In [0]:
class Model(nn.Module):
    def __init__(self, h1_dim, h2_dim):
        super(Model, self).__init__()
        self.first_layer = nn.Linear(input_dim, h1_dim)
        self.second_layer = nn.Linear(h1_dim, h2_dim)
        self.output_layer = nn.Linear(h2_dim, output_dim)
        self.act_func = nn.ReLU()

    def forward(self, input):
        x = self.act_func(self.first_layer)
        x = self.act_func(self.second_layer)
        x = self.act_func(self.output_layer)
        return x